<a href="https://colab.research.google.com/github/pdrobny/Potential_Talents/blob/main/API_groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import warnings
import logging
import random
import requests
import sys
import re
from groq import Groq
from google.colab import userdata
warnings.filterwarnings('ignore', category=UserWarning)

In [4]:
df = pd.read_csv('talents.csv')
df

,id,title,sentence_bert_cossim
0,1,innovative and driven professional seeking a r...,1.000000
1,431,aspiring data science professional focused on ...,0.769162
2,544,data analyst data scientist business analyst d...,0.768222
3,833,data analyst turning complex data into actiona...,0.745245
4,199,ms in information systems northeastern univers...,0.727268
...,...,...,...
1260,648,research specialist university of rochester di...,0.079923
1261,730,medical biller at brick pediatric group,0.072848
1262,990,ingeniero elctrico,0.067254
1263,296,company owner at armstrong cleans carpets,0.056890


In [25]:
# prompt: random sample of 200 from df then create a string of candidates with id and title

random_sample_df = df.sample(n=200, random_state=42)
candidates = ""
for index, row in random_sample_df.iterrows():
  candidate_id = row['id']
  candidate_title = row['title']
  candidates += f"ID: {candidate_id}, Title: {candidate_title}\n"
candidates

'ID: 1233, Title: graduate students of mathematics at university of ioannina uoi mathematician\nID: 639, Title: ms-isom graduate student univerisity of florida  warrington college of business\nID: 332, Title: comp sci mathematics student fourth year at st. josephs university new york data analyst seeking 2025 internshipentry-level opportunities completed google data analyst cert completed ibm data scientist cert\nID: 336, Title: data research assistant at the new york state office of mental health graduate student at the university at albany masters in data science\nID: 509, Title: masters in finance  fintech baruch software engineer data scientist be jadavpur university\nID: 452, Title: crm analyst email lifecycle marketing data-driven\nID: 1054, Title: data scientist with experience in data extraction modeling and visualization. proven expertise in machine learning predictive modeling and transforming data into actionable insights.\nID: 873, Title: data development\nID: 1272, Title: 

In [20]:

# Get a sample of available job titles and job ids to reduce prompt size
# You can adjust the sample size (e.g., 100 or 200) based on testing to stay within the token limit
sample_size = 200
sample_cand = df['title'].sample(min(sample_size, len(df))).tolist()
sample_ids = df['id'].sample(min(sample_size, len(df))).tolist()

# join ids and titles to system knows to keep these together.
candidates2 = "\n".join([f"{job_id}: {job_title}" for job_id, job_title in zip(sample_ids, sample_titles)])
type(candidates2)

str

In [7]:
# Initialize Groq LLM (replace with actual API call)
client= Groq(api_key=userdata.get("GROQ_KEY"))

In [8]:
# replace search term with target job title
search_term = "Data Analyst"

In [33]:
%%time

# Construct prompt
prompt = f"""
Return a list of the top 5 job candidates with full unmodified job title and matching job id from a job titles list ranked by their similirality to the search term in desecnding order.  Only show the answer. Do not reason or explain.
**Search term**
{search_term}

**Job candidates (ID: Title)**
{candidates}


Show answer in following format:
Rank ID   Title
1 - 1: innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.
2 - ...
3 - ...
...
Answer: Top 5 are:
"""

# Call Groq API
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": "You are an AI that ONLY outputs a list of top 5 candidates in strict numbered format. Do NOT explain, think, or reason."},
        {"role": "user", "content": prompt}
    ]
)

# Extract ranked job titles
top_5_jobs = response.choices[0].message.content.split("\n")[:5]

#print(top_5_jobs)
top_5_jobs


CPU times: user 26.4 ms, sys: 3.02 ms, total: 29.4 ms
Wall time: 2.08 s


['1 - 332: comp sci mathematics student fourth year at st. josephs university new york data analyst seeking 2025 internshipentry-level opportunities completed google data analyst cert completed ibm data scientist cert',
 '2 - 1149: machine learning engineer collov ai ml development ai products',
 '3 - 332: comp sci mathematics student fourth year at st. josephs university new york data analyst seeking 2025 internshipentry-level opportunities completed google data analyst cert completed ibm data scientist cert - Replaced with - ',
 '2 - 1136: data engineer cloud platforms aws azure snowflake etlelt automation data pipelines analytics power bi tableau python sql pyspark',
 '3 - 332: comp sci mathematics student fourth year at st. josephs university new york data analyst seeking 2025 internshipentry-level opportunities completed google data analyst cert completed ibm data scientist cert - Replaced with - ']

In [36]:

# Construct strict output prompt
prompt = f"""
Return a list of the top 5 job candidates with full unmodified job title and matching job id from a job titles list ranked by their similirality to the search term in desecnding order.  Only show the answer. Do not reason or explain.
**Search term**
{search_term}

**Job candidates (ID: Title)**
{candidates}


Show answer in following format:
Rank Job ID   Job Title
1 - 1: innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.
2 - ...
3 - ...
...
Answer: Top 5 are:
"""



# Call Groq API
response = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[
        {"role": "system", "content": "You are an AI that ONLY outputs a list of top 5 job titles in strict numbered format. Do NOT explain, think, or reason."},
        {"role": "user", "content": prompt}
    ]
)


# Extract ranked job titles
output = response.choices[0].message.content.strip().split("\n")
top_5_jobs = [line for line in output if re.match(r"^\d\.\s", line)][:5]

for job in top_5_jobs:
    print(job)

1. **ID: 332** - "comp sci mathematics student fourth year at st. josephs university new york data analyst seeking 2025 internshipentry-level opportunities completed google data analyst cert completed ibm data scientist cert"
2. **ID: 336** - "data research assistant at the new york state office of mental health graduate student at the university at albany masters in data science"
3. **ID: 922** - "data analyst at hcn power bi certified indiana university ms ds"
4. **ID: 7** - "data analyst business analyst python snowflake sql machine learning power bi tableau equipped with analytics driven by insights and passionate about impactful solutions."
5. **ID: 452** - "crm analyst email lifecycle marketing data-driven"
